In [1]:
!pip install git+https://github.com/openai/whisper.git


  Cloning https://github.com/openai/whisper.git to c:\users\mario\appdata\local\temp\pip-req-build-za0g2dx9
  Resolved https://github.com/openai/whisper.git to commit 1cea4357687b676b293cb5473e1ade25f5b1cef7
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached numba-0.58.1-cp311-cp311-win_amd64.whl (2.6 MB)
  Using cached tiktoken-0.5.1-cp311-cp311-win_amd64.whl (759 kB)
  Created wheel for openai-whisper: filename=openai_whisper-20231106-py3-none-any.whl size=810671 sha256=cee93da656846175680916ab8b222c53934e6d3e8cf4b676bc2e2bf9b5f5235d
  Stored in directory: C:\Users\Mario\AppData\Local\Temp\pip-ephem-wheel-cache-a96xdaeu\wheels\1f\1d\98\9583695e6695a6ac0ad42d87511097dce5ba486647dbfecb0e
Successfu

  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git 'C:\Users\Mario\AppData\Local\Temp\pip-req-build-za0g2dx9'

[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install sounddevice


     ------------------------------------- 199.7/199.7 kB 11.8 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav

fs = 44100  # Sample rate
seconds = 5  # Duration of recording

print("Recording...")
myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=2)
sd.wait()  # Wait until recording is finished
print("Recording finished")

# Save as WAV file 
wav.write('output.wav', fs, myrecording)  # Save as WAV file 


Recording...
Recording finished


In [51]:
import whisper

model = whisper.load_model("base")  # or another model size based on your preference
result = model.transcribe("output.wav")
print(result["text"])


 Hola, estoy tratando de grabar porque es una prueba.


In [14]:
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav
import queue

fs = 44100  # Sample rate
chunk_size = 1024  # Size of each audio chunk
silence_threshold = 0.02  # Adjust this based on your noise conditions
silence_duration = 2  # Duration of silence in seconds to stop recording
recording_timeout = 20  # Maximum recording duration in seconds

# Queue to hold audio chunks
audio_queue = queue.Queue()

# Callback function to capture audio chunks
def callback(indata, frames, time, status):
    audio_queue.put(indata.copy())

# Open a stream for continuous audio input
stream = sd.InputStream(callback=callback, channels=2, samplerate=fs, blocksize=chunk_size)
stream.start()

# Variables for managing recording state
recording = False
data = []
silence_counter = 0

print("Monitoring for voice activity...")

try:
    while True:
        chunk = audio_queue.get()
        if np.max(np.abs(chunk)) > silence_threshold:
            if not recording:
                print("Starting recording...")
                recording = True
                data = [chunk]
            else:
                data.append(chunk)
            silence_counter = 0
        elif recording:
            silence_counter += chunk_size / fs
            data.append(chunk)
            if silence_counter >= silence_duration:
                print("Stopping recording due to silence.")
                break
except KeyboardInterrupt:
    print("\nRecording stopped manually.")

stream.stop()
stream.close()

# Combine chunks into a single array and save as WAV file
if data:
    recording_data = np.concatenate(data, axis=0)
    wav.write('output.wav', fs, recording_data)
    print("Recording saved as 'output.wav'")
else:
    print("No voice activity detected.")


Monitoring for voice activity...
Starting recording...
Stopping recording due to silence.
Recording saved as 'output.wav'


### Add sound

In [63]:
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav
import queue

fs = 44100  # Sample rate
chunk_size = 1024  # Size of each audio chunk
silence_threshold = 0.02  # Adjust this based on your noise conditions
silence_duration = 2  # Duration of silence in seconds to stop recording
recording_timeout = 10  # Maximum recording duration in seconds

duration = 0.2  # seconds
frequency = 440  # Hz
t = np.linspace(0, duration, int(fs * duration), endpoint=False)
beep_sound = np.sin(2 * np.pi * frequency * t)

# Load start and end sounds
start_sound, _ = wav.read('start_sound.wav')
end_sound, _ = wav.read('end_sound.wav')

# Function to play a sound
def play_sound(sound):
    sd.play(sound, fs)
    sd.wait()

# Queue to hold audio chunks
audio_queue = queue.Queue()

# Callback function to capture audio chunks
def callback(indata, frames, time, status):
    audio_queue.put(indata.copy())

# Open a stream for continuous audio input
stream = sd.InputStream(callback=callback, channels=2, samplerate=fs, blocksize=chunk_size)
stream.start()

# Variables for managing recording state
recording = False
data = []
silence_counter = 0

print("Monitoring for voice activity...")


try:
    while True:
        chunk = audio_queue.get()
        if np.max(np.abs(chunk)) > silence_threshold:
            if not recording:
                print("Starting recording...")
                 # Play start sound
                play_sound(beep_sound)
                recording = True
                data = [chunk]
            else:
                data.append(chunk)
            silence_counter = 0
        elif recording:
            silence_counter += chunk_size / fs
            data.append(chunk)
            if silence_counter >= silence_duration:
                print("Stopping recording due to silence.")
                play_sound(beep_sound)  # Play end sound
                break
except KeyboardInterrupt:
    print("\nRecording stopped manually.")

stream.stop()
stream.close()

# Combine chunks into a single array and save as WAV file
if data:
    recording_data = np.concatenate(data, axis=0)
    wav.write('output.wav', fs, recording_data)
    print("Recording saved as 'output.wav'")
else:
    print("No voice activity detected.")


Monitoring for voice activity...
Starting recording...
Stopping recording due to silence.
Recording saved as 'output.wav'


In [55]:
import os

file_path = "output.wav"

if os.path.exists(file_path):
    print(f"The file {file_path} exists.")
else:
    print(f"The file {file_path} does not exist.")


The file output.wav exists.


In [36]:
# Load start and end sounds
start_sound, _ = wav.read('start_sound.wav')
end_sound, _ = wav.read('end_sound.wav')

# Function to play a sound
def play_sound(sound):
    sd.play(sound, fs)
    sd.wait()
    
play_sound(start_sound)

In [28]:
!pip install sounddevice


In [33]:
import wave

file_path = 'end_sound.wav'  # Replace with the actual path to your WAV file

with wave.open(file_path, 'rb') as wf:
    fs = wf.getframerate()
    print(f"The sampling rate of {file_path} is {fs} Hz.")


The sampling rate of end_sound.wav is 44100 Hz.


In [37]:
import sounddevice as sd
import scipy.io.wavfile as wav

# Load start and end sounds
fs, start_sound = wav.read('start_sound.wav')
_, end_sound = wav.read('end_sound.wav')

# Function to play a sound
def play_sound(sound, fs):
    try:
        sd.play(sound, fs)
        sd.wait()
    except Exception as e:
        print(f"Error: {e}")

play_sound(start_sound, fs)


In [49]:
import numpy as np
import sounddevice as sd

# Generate a test sound (1-second beep at 440 Hz)
fs = 44100  # Sample rate
duration = 0.2  # seconds
frequency = 440  # Hz
t = np.linspace(0, duration, int(fs * duration), endpoint=False)
beep_sound = np.sin(2 * np.pi * frequency * t)

def play_sound(end_sound, fs):
    try:
        sd.play(end_sound, fs)
        sd.wait()
    except Exception as e:
        print(f"Error: {e}")

play_sound(beep_sound, fs)


In [58]:
frequency_start = 440  # Hz
frequency_end = 880  # Hz
chirp_sound = np.sin(2 * np.pi * np.linspace(frequency_start, frequency_end, int(fs * duration)) * t)
play_sound(chirp_sound)


In [61]:
fs = 44100  # Sample rate
duration = 0.1  # Short duration for a quick click sound

# Generate a click sound
t = np.linspace(0, duration, int(fs * duration), endpoint=False)
click_sound = np.sin(2 * np.pi * 1000 * t) * np.exp(-t * 50)

play_sound(click_sound)